In [ ]:
##### Import important modules #####

from os.path import join as opj
from nipype.interfaces.spm import Level1Design,EstimateModel, EstimateContrast
import nipype.interfaces.spm as spm
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.pipeline.engine import Workflow, Node, MapNode


In [ ]:
### path to spm standalone and MCR
matlab_cmd = '/usr/local/spm12/run_spm12.sh /usr/local/MCR/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [ ]:
##### Specify interface behaviors #####

# Matlab - Specify path to current SPM and the MATLAB's default mode, comment these lines if you're using the standalone version of SPM
#from nipype.interfaces.matlab import MatlabCommand
#MatlabCommand.set_default_paths('/usr/local/MATLAB/R2016b/toolbox/spm12/') # set the path to your local spm folder
#MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

In [ ]:
experiment_dir = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/'
output_dir = 'derivatives/1stlevel/output_1stlevel_allcond'
working_dir = 'derivatives/1stlevel/workingdir_1stlevel_allcond'
input_dir_preproc = 'derivatives/preprocessing/output_preproc'

In [ ]:
# list of subjects
subject_list = ['sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22', 'sub-23', 'sub-24'] 

TR = 1.45   

In [ ]:
##### Create & specify nodes to be used and connected during the 1st level pipeline #####

# SpecifyModel - Generates SPM-specific Model
modelspec = Node(SpecifySPMModel(concatenate_runs=False,
                                 input_units='secs',
                                 output_units='secs',
                                 time_repetition=TR,
                                 high_pass_filter_cutoff=128),
                 name="modelspec")

# Level1Design - Generates an SPM design matrix
level1design_masks = Node(Level1Design(bases={'hrf': {'derivs': [0, 0]}},
                                 timing_units='secs',
                                 interscan_interval=TR,
                                 model_serial_correlations='AR(1)'),
                    name="level1design")

# EstimateModel - estimate the parameters of the model
level1estimate = Node(EstimateModel(estimation_method={'Classical': 1}),
                      name="level1estimate")

# EstimateContrast - estimates contrasts
conestimate = Node(EstimateContrast(), name="conestimate")

In [ ]:
# Initiation of the 1st-level analysis workflow
l1analysis = Workflow(name='l1analysis')
l1analysis.base_dir = opj(experiment_dir, working_dir)

# Connect up the 1st-level analysis components
l1analysis.connect([(modelspec, level1design_masks, [('session_info',
                                                'session_info')]),
                    (level1design_masks, level1estimate, [('spm_mat_file',
                                                     'spm_mat_file')]),
                    #(level1estimate, conestimate, [('spm_mat_file',
                    #                                'spm_mat_file'),
                    #                               ('beta_images',
                    #                                'beta_images'),
                     #                              ('residual_image',
                    #                                'residual_image')]),
                    ])
                                    

In [ ]:
from nipype.interfaces.base import Bunch
def get_subject_info(subject_id):
    from os.path import join as opj
    import pandas as pd
    from nipype.interfaces.base import Bunch
    path = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/%s/func'%subject_id
    for run in ['1', '2', '3', '4']:
        file = pd.read_table(opj(path, subject_id+'_task-tnac_'+'run-'+run+'_events.tsv'))
        # sort stimuli/rows so that they are always in the same order
        file['stimuli_cat'] = pd.Categorical(file['block_modality'], categories=['block_music', 'block_song', 'block_voice'], ordered=True)
        sorted_file = file.sort_values('stimuli_cat')

        # create lists for each run including the stimuli and their particular onset times
        onset_array = []
        duration_array = []
        stimuli_array = []
        for group in sorted_file.groupby('block_modality', sort=False):
            stimuli_array.append(group[0])
            onset_array.append(list(round(group[1].corr_block_start,1)))
            duration_array.append(list(round(group[1].block_duration,1)))
            
        all_cond_onset = onset_array[0] + onset_array[1] + onset_array[2] 
        all_cond_dur = duration_array[0] + duration_array[1] + duration_array[2]    
        onset_array.append(all_cond_onset)
        duration_array.append(all_cond_dur)
        stimuli_array.append("all_cond")
    
        if run == '1':
            info_run1 = [stimuli_array, onset_array, duration_array]
        elif run == '2':
            info_run2 = [stimuli_array, onset_array, duration_array]
        elif run == '3':
            info_run3 = [stimuli_array, onset_array, duration_array]
        elif run == '4':
            info_run4 = [stimuli_array, onset_array, duration_array]
        else:
            print(info_run_error)

    # create subject_info including all runs
    subject_info=[]
    for r in range(4):
        if r == 0:
            subject_info.insert(r,
                                (Bunch(conditions=info_run1[0],
                                 onsets = info_run1[1],
                                 durations = info_run1[2],
                                 amplitudes=None,
                                 tmod=None,
                                 pmod=None)))   
        elif r == 1:
            subject_info.insert(r,
                                (Bunch(conditions=info_run2[0],
                                 onsets = info_run2[1],
                                 durations = info_run2[2],
                                 amplitudes=None,
                                 tmod=None,
                                 pmod=None))) 
        elif r == 2:
            subject_info.insert(r,
                                (Bunch(conditions=info_run3[0],
                                 onsets = info_run3[1],
                                 durations = info_run3[2],
                                 amplitudes=None,
                                 tmod=None,
                                 pmod=None)))
        elif r == 3:
            subject_info.insert(r,
                                (Bunch(conditions=info_run4[0],
                                 onsets = info_run4[1],
                                 durations = info_run4[2],
                                 amplitudes=None,
                                 tmod=None,
                                 pmod=None)))

        else: 
            print("error2")          
            
    return(subject_info)

In [ ]:
# get subject info - get subject specific condition information
getsubjectinfo = Node(Function(input_names=['subject_id'],
                               output_names=['subject_info'],
                               function=get_subject_info),
                      name='getsubjectinfo')

### Define model specific contrasts

# condition names
condition_names = ['block_music', 'block_song', 'block_voice', 'all_cond'] # enter the condition names of your study

# contrasts; name your contrasts, specify if T or F, enter contrast vector (also the 6 motion-parameters--> 6 zeros more!)
contrast01 = ['block_music', 'T', condition_names, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]*4]
contrast02 = ['block_song', 'T', condition_names, [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]*4]
contrast03 = ['block_voice', 'T', condition_names, [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]*4]
contrast04 = ['all_conditions', 'T', condition_names, [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]*4]
#contrast05 = ['activation', 'F', [contrast01, contrast02, contrast03]]

contrast_list = [contrast01, contrast02, contrast03, contrast04]#, contrast05]


In [ ]:
##### Specify input and output stream #####

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id',
                                            'contrasts'],
                                    contrasts=contrast_list),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
templates = {'func_smooth': opj(input_dir_preproc, 'smooth', '{subject_id}',  'sr{subject_id}_*.nii'),
             'realign_par': opj(input_dir_preproc, 'realign', '{subject_id}', 'rp_{subject_id}_*.txt'),
             'mask': opj(input_dir_preproc, 'masks', '{subject_id}', 'aparc_robust_BET.nii')}
selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir),
                   name="selectfiles")
       

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', '')]
datasink.inputs.substitutions = substitutions

In [ ]:
##### establish input and output streams by connecting Infosource, SelectFiles and DataSink to the main workflow #####
l1analysis.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (selectfiles, modelspec,[('func_smooth', 'functional_runs')]),  
                  (selectfiles, modelspec,[('realign_par', 'realignment_parameters')]),
                  (selectfiles, level1design_masks, [('mask', 'mask_image')]),
                  (infosource, getsubjectinfo, [('subject_id', 'subject_id')]),
                  (getsubjectinfo, modelspec, [('subject_info',
                                                 'subject_info')]),
                  #(infosource, conestimate, [('contrasts',
                  #                           'contrasts')]),
                  (level1estimate, datasink, [('mask_image', 'contrasts.@mask'),
                                              ('spm_mat_file','contrasts.@spm_mat')]), 
                  #(conestimate, datasink, [('spm_mat_file',
                  #                         'contrasts.@spm_mat'),
                  #                        ('spmT_images',
                  #                         'contrasts.@T'),
                   #                       ('con_images',
                   #                        'contrasts.@con')]),
                   ])  

In [ ]:
#### visualize the pipeline ####

# Create a colored mvpaflow output graph
l1analysis.write_graph(graph2use='colored',format='png', simple_form=True)

# Create a detailed mvpaflow output graph
l1analysis.write_graph(graph2use='flat',format='png', simple_form=True)


In [ ]:
# Visualize graphs
from IPython.display import Image
Image(filename='/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/1stlevel/workingdir_1stlevel/l1analysis/graph.png')

In [ ]:
Image(filename='/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/1stlevel/workingdir_1stlevel/l1analysis/graph_detailed.png')

In [ ]:
#### run the workflow using multiple cores ####
l1analysis.run('MultiProc', plugin_args={'n_procs':4})

In [ ]:
!tree /media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/1stlevel/output_1stlevel_allcond/